In [5]:
import numpy as np

In [6]:
# Hamming(7,4) matrices
G = np.array([
    [1, 0, 0, 0, 0, 1, 1],
    [0, 1, 0, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 1, 0],
    [0, 0, 0, 1, 1, 1, 1]
])

H = np.array([
    [0, 1, 1, 1, 1, 0, 0],
    [1, 0, 1, 1, 0, 1, 0],
    [1, 1, 0, 1, 0, 0, 1]
])

syndrome_table = {
    (0, 0, 0): None,
    (0, 0, 1): 6,
    (0, 1, 0): 5,
    (0, 1, 1): 2,
    (1, 0, 0): 4,
    (1, 0, 1): 0,
    (1, 1, 0): 1,
    (1, 1, 1): 3,
}

In [7]:
# Batch encoder: (n x 4) → (n x 7)
def encode_batch(data_bits):
    return np.dot(data_bits, G) % 2

# Batch decoder: (n x 7) → (n x 4)
def decode_batch(received_batch):
    decoded = []
    for received in received_batch:
        syndrome = np.dot(H, received.T) % 2
        error_bit = syndrome_table.get(tuple(syndrome))
        corrected = received.copy()
        if error_bit is not None:
            corrected[error_bit] ^= 1
        decoded.append(corrected[:4])
    return np.array(decoded)

# Gaussian channel for a batch
def gaussian_channel_batch(codewords, noise_std=0.5):
    noise = np.random.normal(0, noise_std, codewords.shape)
    received = codewords + noise
    return np.round(received).astype(int) % 2

# Uniform (bit-flip) channel for a batch
def uniform_channel_batch(codewords, flip_prob=0.1):
    flips = np.random.rand(*codewords.shape) < flip_prob
    return (codewords ^ flips.astype(int))

# Bit Error Rate
def bit_error_rate(original, decoded):
    xor = original ^ decoded
    errors = np.sum(xor)
    total = original.size
    return errors / total, xor

In [13]:
# Generate n random 4-bit messages
n = 3
msgs = np.random.randint(0, 2, size=(n, 4))

print("Original messages:\n", msgs)

# Encode
codewords = encode_batch(msgs)

# Pass through Gaussian channel
noisy_g = gaussian_channel_batch(codewords, noise_std=0.5)
print("\nReceived codewords (Gaussian Noise):\n", noisy_g)

# Decode
decoded_g = decode_batch(noisy_g)
print("\nDecoded messages:\n", decoded_g)

# Compute Bit Error Rate
ber_g, _ = bit_error_rate(msgs, decoded_g)
print("\nBit Error Rate (Gaussian):", ber_g)

# Pass through Uniform channel
noisy_u = uniform_channel_batch(codewords, flip_prob=0.2)
print("\nReceived codewords (Uniform Noise):\n", noisy_u)

# Decode
decoded_u = decode_batch(noisy_u)
print("\nDecoded messages:\n", decoded_u)

# Compute Bit Error Rate
ber_u, _ = bit_error_rate(msgs, decoded_u)
print("\nBit Error Rate (Uniform):", ber_u)

Original messages:
 [[0 1 1 0]
 [0 0 0 1]
 [1 0 0 0]]

Received codewords (Gaussian Noise):
 [[1 1 0 1 1 1 1]
 [0 0 0 0 0 1 1]
 [1 0 0 0 0 1 1]]

Decoded messages:
 [[1 0 0 1]
 [0 0 1 0]
 [1 0 0 0]]

Bit Error Rate (Gaussian): 0.5

Received codewords (Uniform Noise):
 [[0 1 1 0 0 0 1]
 [0 0 0 0 0 1 1]
 [1 0 0 0 0 0 1]]

Decoded messages:
 [[0 1 1 0]
 [0 0 1 0]
 [1 0 0 0]]

Bit Error Rate (Uniform): 0.16666666666666666
